<a href="https://colab.research.google.com/github/statistics-jun/2022-1-ESAA/blob/main/ESAA_Week11_Recommendation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2022-1 ESAA 5기 김혜준 개념 필사 - 0520(금) 11주차 과제
 
## <파이썬 머신러닝 완벽 가이드>
## Chapter 9 추천시스템

### 08 파이썬 추천 시스템 패키지 - Surprise

#### Surprise 패키지 소개
* 파이썬 기반 추천 시스템 구축을 위한 전용 패키지
* 주요 장점
  - 다양한 추천 알고리즘
    - 사용자 또는 아이템 기반 이웃 협업 필터링, SVD, SVD++, NMF 기반 잠재 요인 협업 필터링
  - 사이킷런의 핵심 API와 유사한 API명으로 작성
    - fit(), predict(), train_test_split(), cross_validate(), GridSearchCV 클래스



In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633723 sha256=e9fd3c1874b12c26ecab2a312cd3821a32812e7897830f25ed46d15e296926eb
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


#### Surprise를 이용한 추천 시스템 구축

In [ ]:
import surprise

##### Surprise 관련 모듈 임포트

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

##### 데이터 로딩
* `Dataset` 클래스를 이용해서만 가능
* 로우 레벨 형태인 데이터만 처리
  - 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하기 때문
  

In [ ]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


> `load_builtin()`의 ml-100k : 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트
  * 칼럼 분리 문자가 탭(\t)

> cf) 무비렌즈 사이트에서 직접 내려받은 데이터 : 최근 영화에 대한 평점 정보 有
  * 칼럼 분리 문자가 콤마(,)인 csv 파일

##### SVD 행렬 분해를 통한 잠재 요인 협업 필터링 수행

###### 1. 학습 데이터 세트 기반으로 추천 알고리즘 학습
  * `fit(학습 데이터 세트)`

In [ ]:
algo = SVD() # 알고리즘 객체 생성
algo.fit(trainset) # 추천 알고리즘 학습

###### 2. 학습된 추천 알고리즘 기반으로 테스트 데이터 세트에 대해 추천 수행
  * Surprise의 추천 예측 메서드 
    - `test()` : 사용자-아이템 평점 데이터 세트 전체에 대해 추천 예측 -> 입력된 데이터 세트에 대한 추천 데이터 세트 생성
    - `predict()` : 개별 사용자의 아이템에 대한 추천 평점 예측

1) `test(테스트 데이터 세트)`

In [ ]:
predictions = algo.test(testset) # 테스트 데이터 전체에 대해 추천 영화 평점 데이터 생성
print('prediction type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6329782179903294, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8991921531138827, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.031227242951512, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6426259934913623, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.807891999910418, details={'was_impossible': False})]

> *predictions* : SVD 알고리즘 객체의 `test()` 호출 결과
  - type : list
    - list 객체 : 25,000개의 `Prediction` 객체 가짐
      - `prediction` 객체 : Surprise 패키지에서 제공하는 데이터 타입
        - 개별 사용자 아이디(`uid`), 영화(또는 아이템) 아이디(`iid`), 실제 평점(`r_ui`) 정보 기반 추천 예측 평점 데이터(`est`)를 튜플 형태로 가짐
        - `details` 속성 : 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는 데 사용
          - 'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미
  - size : 25,000(입력 인자 데이터 세트 크기와 동일)

In [ ]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ] # 3개의 Prediction 객체에서 uid, iid, est 속성 추출

[('120', '282', 3.6329782179903294),
 ('882', '291', 3.8991921531138827),
 ('535', '507', 4.031227242951512)]

2) `predict(str(개별 사용자 아이디), str(아이템 아이디))` 

In [ ]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
# 기존 평점 정보(r_ui)는 선택 사항
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid) # 개별 사용자의 아이템에 대하여 추천 예측 평점을 포함한 정보 반환
print(pred)

user: 196        item: 302        r_ui = None   est = 4.25   {'was_impossible': False}


> *pred* : SVD 알고리즘 객체의 `predict()` 호출 결과
  - `est` : 추천 예측 평점

###### 3. 테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이 평가
  * `accuracy` 모듈 : RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보 제공

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9490


0.9489928434875834

#### Surprise 주요 모듈 소개


##### Dataset
* 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 반드시 되어 있어야 함(네 번째 칼럼부터는 아예 로딩을 수행하지 않음)
* API
  - `Dataset.load_builtin(name='ml-100k')` : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 다운로드
    - `name` : 대상 데이터(ml-100k, ml-1M)
  - `Dataset.load_from_file(file_path, reader)` : 콤마, 탭 등으로 칼럼이 분리된 포맷의 OS 파일에서 데이터 로딩
    - `file_path` : OS 파일명(경로)
    - `reader` : 파일 포맷 지정
  - `Dataset.load_from_df(df, reader)` : 판다스의 DataFrame에서 데이터 로딩
    - `df` : DataFrame 객체
    - `reader` : 파일 포맷 지정
  

##### OS 파일 데이터를 Surprise 데이터 세트로 로딩

* 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안 됨

In [ ]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/ratings.csv')
# ratings_noh.csv 파일로 업로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/ratings_noh.csv', index=False, header=False)

> rating_noh.csv : ratings.csv 파일에서 헤더가 삭제된 파일

* 로딩 전 `Reader` 클래스를 이용해 데이터 파일의 파싱 포맷을 정의해야 함
  - `Reader` 클래스의 주요 생성 파라미터
    - `line_format` *(string)* : 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식
    - `sep='\t'` *(char)* : 칼럼을 분리하는 분리자, DataFrame에서 입력받을 경우에는 기재할 필요 없음
    - `rating_scale=(1, 5)` *(tuple, optional)* : 평점 값의 최소 ~ 최대 평점 설정
  - `Reader` 객체 생성 : `Reader(line_format='각 필드의 칼럼명', sep='칼럼 분리문자', rating_scale=(최소 평점, 최대 평점))`

In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5)) # reader 객체 생성
data = Dataset.load_from_file('/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/ratings_noh.csv', reader=reader) # 생성된 reader 객체 참조해 데이터 파일을 파싱하면서 로딩

> *reader* 객체 생성
  - user, item, rating, timestamp 4개의 칼럼으로 데이터가 구성되어 있음을 명시
  - 각 칼럼의 분리문자는 콤마
  - 평점 단위는 0.5, 최대 평점는 5

In [ ]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0) # n_factors는 잠재 요인 크기 K값을 지정

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

##### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩



In [ ]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/ratings.csv') # ratings.csv 파일을 DataFrame으로 로딩
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### Surprise 추천 알고리즘 클래스
* SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
* KNNBasic : 최근점 이웃 협업 필터링을 위한 KNN 알고리즘
* BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

* `SVD` 클래스
  - 비용 함수 : 사용자 베이스라인 편향성을 감안한 평점 에측에 규제를 적용한 것
  - 입력 파라미터
     - `n_factors=100` : 잠재 요인 K의 개수, 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생
     - `n_epochs=20` : SGD(Stochastic Gradient Descent) 수행 시 반복 횟수
     - `biased=True` *(bool)* : 베이스라인 사용자 편향 적용 여부
     
    -> 주로 `n_factors`와 `n_epochs`를 변경해 튜닝할 수 있으나 효과는 크지 않고, `biased`는 큰 이슈가 없는 한 디폴트인 True로 설정 유지하는 것이 좋음

* 시간이 너무 오래 걸려 큰 데이터에 사용하기 어려운 SVD++를 제외하면 SVD, k-NN Baseline이 가장 성능 평가 수치 좋음

#### 베이스라인 평점
* 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 방식
* 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산
  - 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
  - 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
  - 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

#### 교차 검증과 하이퍼 파라미터 튜닝
* `surprise.model_selection.cross_validate(알고리즘 객체, 데이터, measures=[성능 평가 방법], cv=폴드 데이터 세트 개수)` : 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차 검증 수행하는 함수

In [ ]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) # 5개의 학습/검증 폴드 데이터 세트로 분리해 교차 검증 수행

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8717  0.8751  0.8743  0.8680  0.8812  0.8741  0.0043  
MAE (testset)     0.6700  0.6717  0.6754  0.6650  0.6760  0.6716  0.0040  
Fit time          8.72    5.23    5.30    5.17    5.24    5.93    1.39    
Test time         0.15    0.30    0.16    0.28    0.16    0.21    0.07    


{'fit_time': (8.717796087265015,
  5.2296226024627686,
  5.30039119720459,
  5.174804449081421,
  5.239276647567749),
 'test_mae': array([0.66999426, 0.67172334, 0.67543691, 0.66502915, 0.67599948]),
 'test_rmse': array([0.8716714 , 0.87509104, 0.87429327, 0.86804717, 0.88124313]),
 'test_time': (0.1523606777191162,
  0.2971456050872803,
  0.15798735618591309,
  0.28430676460266113,
  0.1612379550933838)}

> 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여줌

* `surprise,model_selection.GridSearchCV` 클래스 : 교차 검증을 통한 하이퍼 파라미터 최적화 수행
  1. `GridSearchCV` 객체 *gs* 생성 : `gs = GridSearchCV(알고리즘 유형, 최적화할 파라미터의 딕셔너리, measures=[성능 평가 방법], cv=폴드 세트 개수)`
  2. `gs.fit(데이터)`로 학습 수행
  3. `gs`의 `best_score[성능 평가 방법]`과 `best_params[성능 평가 방법]`으로 최고 성능 평가 수치와 최적 하이퍼 파라미터 확인

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mse'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8783453537982503
{'n_epochs': 20, 'n_factors': 50}


> 'n_epochs': 20, 'n_factors': 50일 때 3개 폴드의 검증 데이터 세트에서 최적 RMSE는 0.878

#### Surprise를 이용한 개인화 영화 추천 시스템 구축

##### 잠재 요인 협업 필터링 기반 개인화된 영화 추천 구현
* 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천

###### ratings.csv 데이터 전체를 학습 데이터로 생성

* Surprise는 데이터 세트를 `train_test_split()`을 이용해 내부에서 사용하는 `TrainSet` 클래스 객체로 변환하지 않으면 `fit()`을 통해 학습할 수 없음

In [ ]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

* `DatasetAutoFolds` 클래스 : 데이터 세트 전체를 학습 데이터로 생성
  1. `DatasetAutoFolds` 객체 *data_folds* 생성 : `data_folds = DatasetAutoFolds(ratings_file='파일 경로', reader=Reader 객체)`
  2. *data_folds* 객체의 `build_full_trainset()` 호출

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 rating_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

###### SVD를 이용해 학습 수행

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

###### 특정 사용자가 선정된 영화를 관람하였는지 확인

In [ ]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/drive/MyDrive/ESAA/2022-1/과제/개념 필사/0520(금) 11주차 과제_추천시스템(2)/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


> userId 9는 movieId 42를 아직 관람하지 않음


###### `predict()`를 이용해 movieId 42인 영화에 대한 userId 9 사용자의 추천 예상 평점 계산

In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


> 추천 예측 평점(`est`) : 3.13

###### 추천 대상이 되는 영화를 추출
* 사용자가 평점을 매기지 않은 모든 영화 정보를 추출하여 반환하는 함수 `get_unseen_surprise` 생성

In [ ]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력갑으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


> userId 9번은 전체 9742개 영화 중 46개만 평점 부여해 추천 대상 영화는 9696개

###### SVD를 이용해 높은 예측 평점을 가진 순으로 영화 추천
* 함수 `recomm_movie_by_surprise` 생성
  - 입력 인자 : 학습이 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화 리스트 객체, 추천 상위 N개 개수
  - 반환값 : TOP-N 개의 Prediction 객체의 영화 아이디, 영화 제목, 예측 평점 정보

In [ ]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortket_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


> userId 9번에게 Usual Suspects, Godfather, Goodfellas 등의 서스펜스/스릴러/범죄 영화 및 Star Wars와 같은 액션 영화가 주로 추천됨